In [1]:
import pandas as pd
from pathlib import Path

In [21]:
BASE_DIR = Path(r"E:\jupyternoteBookWorkPath\erp\house_eco_poi1")
house_file = BASE_DIR / "shanghai_house_price_9_with_distances.csv" 
macro_file = BASE_DIR / "A_macroeconomic_factors_eng1.xlsx"   
output_file = BASE_DIR / "shanghai_house_price_9_with_macro_3mavg.csv"

In [23]:
# read data
house_df = pd.read_csv(house_file)
macro_df = pd.read_excel(macro_file)

house_df["transaction_date"] = pd.to_datetime(house_df["transaction_date"], errors="coerce")
macro_df["date"] = pd.to_datetime(macro_df["date"], errors="coerce")

In [25]:
# preprocess data
# get 6 macro variable columns except date
macro_vars = [c for c in macro_df.columns if c.lower() != "date"]
macro_df = macro_df[["date"] + macro_vars].dropna(subset=["date"]).sort_values("date")

# unify to "end of month" and keep datetime64[ns] precision:
# period to Timestamp(end), avoid converting to datetime64[s]
macro_df["month_end"] = macro_df["date"].dt.to_period("M").dt.to_timestamp(how="end")
macro_df = (
    macro_df
    .drop(columns=["date"])
    .drop_duplicates(subset=["month_end"])
    .set_index("month_end")
    .sort_index()
)

# calculate the average of the first three months (excluding the transaction month)
macro_roll_3m = macro_df.rolling(3, min_periods=1).mean().shift(1)

# reset index to column for merge_asof
macro_roll_3m = macro_roll_3m.reset_index().rename(columns={"month_end": "macro_month_end"})

# align transaction data with the month and match them
# transaction records are used for matching the end of the month: take the end of the transaction month, then move forward one month.
house_df = house_df.copy()
house_df["tx_month_end"] = house_df["transaction_date"].dt.to_period("M").dt.to_timestamp(how="end")
house_df["lookup_month_end"] = house_df["tx_month_end"] - pd.offsets.MonthEnd(1)

# force both keys to be datetime64[ns] without time zone
house_df["lookup_month_end"]   = pd.to_datetime(house_df["lookup_month_end"],   utc=False).astype("datetime64[ns]")
macro_roll_3m["macro_month_end"] = pd.to_datetime(macro_roll_3m["macro_month_end"], utc=False).astype("datetime64[ns]")

# sorting for merge_asof
house_df = house_df.sort_values("lookup_month_end")
macro_roll_3m = macro_roll_3m.sort_values("macro_month_end")

# asof merge: the most recent month in the right table that is less than or equal to the left key
matched = pd.merge_asof(
    left=house_df,
    right=macro_roll_3m,
    left_on="lookup_month_end",
    right_on="macro_month_end",
    direction="backward"
)

In [26]:
# add suffix _3m_avg to 6 macro variables
rename_map = {col: f"{col}_3m_avg" for col in macro_vars}
matched = matched.rename(columns=rename_map)

# only keep the original fields + add 6 new fields
result_cols = list(house_df.columns) + [f"{c}_3m_avg" for c in macro_vars]
result = matched[result_cols].copy()

# save results
result.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"name：{output_file}")

处理完成：已为每条交易添加“前3个月（不含当月）”的6个宏观变量均值。
输出文件：E:\jupyternoteBookWorkPath\erp\house_eco_poi1\shanghai_house_price_9_with_macro_3mavg.csv
